In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from tqdm.contrib import tmap

# Enable tqdm for Pandas
tqdm.pandas()

/Users/esmirmesic/opt/anaconda3/envs/bem114/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# A

In [2]:
crsp_data = pd.read_csv("data/cleaned_crsp.csv")
crsp_data['date'] = pd.to_datetime(crsp_data['date'])
crsp_data['RET'] = crsp_data['RET'].str.replace('C', '')
crsp_data['RET'] = pd.to_numeric(crsp_data['RET'], errors='coerce')
crsp_data['date'] = pd.to_datetime(crsp_data['date'], format='%Y-%m-%d')

In [3]:
import pandas_datareader as pdr

start_date = '1926-01-01'
end_date = '2020-12-31'


ff5_factors = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start=start_date, end=end_date)[0]
ff5_factors = ff5_factors / 100  # Convert to decimal
ff5_factors.index = ff5_factors.index.to_timestamp('M')  # Convert index to monthly-end dates

ff12 = pdr.get_data_famafrench('12_industry_Portfolios', start=start_date, end=end_date)[0]
ff12 = ff12 / 100
ff12.index = ff12.index.to_timestamp('M')

/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_12833/3641822038.py:7: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff5_factors = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start=start_date, end=end_date)[0]
/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_12833/3641822038.py:7: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff5_factors = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start=start_date, end=end_date)[0]
/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_12833/3641822038.py:11: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or r

In [26]:
import yfinance as yf

top10_holdings = pd.read_csv("data/top10_holdings_brk_arkk.csv")

# Get BRK-A and ARKK data from Yahoo Finance
brk = yf.download("BRK-A", start="1980-01-31", end="2020-12-31", interval="1mo")
brk.index = pd.to_datetime(brk.index)
arkk = yf.download("ARKK", start="2014-10-31", end="2020-12-31", interval="1mo")
arkk.index = pd.to_datetime(arkk.index)

# Make sure the index is datetime
brk.index = pd.to_datetime(brk.index)
arkk.index = pd.to_datetime(arkk.index)

brk.index = brk.index.to_period("M").to_timestamp("M")
arkk.index = arkk.index.to_period("M").to_timestamp("M")

# Calculate monthly stock returns
brk["Return"] = brk["Adj Close"].pct_change()
arkk["Return"] = arkk["Adj Close"].pct_change()

brk = brk.dropna()
arkk = arkk.dropna()

# Estimate the FF5 model for each strategy over their full histories and the same sample period
# Merge data
brk_ff5 = pd.merge(brk, ff5_factors, left_index=True, right_index=True)
arkk_ff5 = pd.merge(arkk, ff5_factors, left_index=True, right_index=True)

# Find the common time period for both stocks
start_date = max(brk_ff5.index.min(), arkk_ff5.index.min())
end_date = min(brk_ff5.index.max(), arkk_ff5.index.max())

# Create the same sample period data
brk_ff5_same_period = brk_ff5.loc[start_date:end_date]
arkk_ff5_same_period = arkk_ff5.loc[start_date:end_date]

# Perform regressions for the same sample period
X_brk_same_period = sm.add_constant(brk_ff5_same_period[["Mkt-RF", "SMB", "HML", "RMW", "CMA"]])
X_arkk_same_period = sm.add_constant(arkk_ff5_same_period[["Mkt-RF", "SMB", "HML", "RMW", "CMA"]])

model_brk_same_period = sm.OLS(brk_ff5_same_period["Return"], X_brk_same_period).fit()
model_arkk_same_period = sm.OLS(arkk_ff5_same_period["Return"], X_arkk_same_period).fit()

# Perform regressions
X_brk = sm.add_constant(brk_ff5[["Mkt-RF", "SMB", "HML", "RMW", "CMA"]])
X_arkk = sm.add_constant(arkk_ff5[["Mkt-RF", "SMB", "HML", "RMW", "CMA"]])

model_brk = sm.OLS(brk_ff5["Return"], X_brk).fit()
model_arkk = sm.OLS(arkk_ff5["Return"], X_arkk).fit()

# Regress returns for each strategy on the Fama French 12 Industry Portfolios
X_brk_ff12 = sm.add_constant(ff12.loc[brk_ff5.index])
X_arkk_ff12 = sm.add_constant(ff12.loc[arkk_ff5.index])

model_brk_ff12 = sm.OLS(brk_ff5["Return"], X_brk_ff12).fit()
model_arkk_ff12 = sm.OLS(arkk_ff5["Return"], X_arkk_ff12).fit()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [27]:
model_arkk.summary(), model_brk.summary()

(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                 Return   R-squared:                       0.814
 Model:                            OLS   Adj. R-squared:                  0.800
 Method:                 Least Squares   F-statistic:                     58.61
 Date:                Sun, 23 Apr 2023   Prob (F-statistic):           3.92e-23
 Time:                        20:34:29   Log-Likelihood:                 136.38
 No. Observations:                  73   AIC:                            -260.8
 Df Residuals:                      67   BIC:                            -247.0
 Df Model:                           5                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------------

In [28]:
model_arkk_ff12.summary(), model_brk_ff12.summary()

(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                 Return   R-squared:                       0.833
 Model:                            OLS   Adj. R-squared:                  0.800
 Method:                 Least Squares   F-statistic:                     25.02
 Date:                Sun, 23 Apr 2023   Prob (F-statistic):           5.99e-19
 Time:                        20:34:29   Log-Likelihood:                 140.43
 No. Observations:                  73   AIC:                            -254.9
 Df Residuals:                      60   BIC:                            -225.1
 Df Model:                          12                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------------

In [25]:
model_arkk_same_period.summary(), model_brk_same_period.summary()

(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                 Return   R-squared:                       0.814
 Model:                            OLS   Adj. R-squared:                  0.800
 Method:                 Least Squares   F-statistic:                     58.61
 Date:                Sun, 23 Apr 2023   Prob (F-statistic):           3.92e-23
 Time:                        20:34:26   Log-Likelihood:                 136.38
 No. Observations:                  73   AIC:                            -260.8
 Df Residuals:                      67   BIC:                            -247.0
 Df Model:                           5                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------------

# B.

Cathie Wood and Warren Buffet do not appear to have similar investment strategies (during the period in which we have data from both). Based on the OLS regression over the FF12 data, the coefficients across their two models vary drastically, indicating that their models are different.

Warren Buffett is more like a value investor due to his positive and statistically significant HML coefficient (both historically and recently, although he has been acting less like a value investor in recent periods, as indicated by his decline in HML). In contrast, Cathie Wood has a negative and statistically significant HML coefficient, indicating that she is acting more like a growth investor than a value investor.

Warren Buffett’s portfolio behaves closest to Consumer Nondurables (NoDur), Shops, Banking Sector (Money), and (Other). Cathie Wood’s portfolio behaves closest to Consumer Durables (Durble), Manufacturing (Manuf), Business Equipment (BusEq), Health (Hlth).

Both Buffett and Wood focus on consumer goods (although different types), and Buffett focuses on Banking. The top 10 holdings focus on banking and consumer goods, so the portfolio behavior analysis tracks.

